# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
#%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    x='Lng',  
    y='Lat',  
    geo=True,  
    tiles='OSM',  
    color='blue',  
    size= "Humidity",
    title='City Locations'
)

# Display the map
humidity_map_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [47]:
# Narrow down cities that fit criteria and drop any results with null values
nice_weather_df = city_data_df[
    (city_data_df['Max Temp'] <= 26) &
    (city_data_df['Humidity'] <=70)
]

# Drop any rows with null values
nice_weather_df = nice_weather_df.dropna()

# Display sample data
nice_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.39,70,54,10.08,TF,1729869587
2,2,olonkinbyen,70.9221,-8.7187,-2.96,55,80,4.54,SJ,1729869590
4,4,balkanabat,39.5108,54.3671,15.72,63,24,3.98,TM,1729869593
5,5,zhangye,38.9342,100.4517,8.41,45,37,1.48,CN,1729869594
6,6,puerto natales,-51.7236,-72.4875,9.25,57,100,18.01,CL,1729869330
...,...,...,...,...,...,...,...,...,...,...
573,573,muroto-misakicho,33.2833,134.1500,22.57,67,100,6.76,JP,1729870373
576,576,heroica ciudad de tlaxiaco,17.2667,-97.6833,19.78,63,0,1.04,MX,1729870377
584,584,viana do castelo,41.6932,-8.8329,14.37,68,91,5.19,PT,1729870389
587,587,al bawiti,28.3492,28.8659,22.58,31,0,4.80,EG,1729870393


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = nice_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,70,
2,olonkinbyen,SJ,70.9221,-8.7187,55,
4,balkanabat,TM,39.5108,54.3671,63,
5,zhangye,CN,38.9342,100.4517,45,
6,puerto natales,CL,-51.7236,-72.4875,57,
...,...,...,...,...,...,...
573,muroto-misakicho,JP,33.2833,134.1500,67,
576,heroica ciudad de tlaxiaco,MX,17.2667,-97.6833,63,
584,viana do castelo,PT,41.6932,-8.8329,68,
587,al bawiti,EG,28.3492,28.8659,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
filters =  f"circle:{longitude},{latitude},{radius}",
bias = f"proximity:{longitude},{latitude}"

params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["categories"] = f"accommodation.hotel"
    params['filter'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary
    names = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = names.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port-aux-francais - nearest hotel: Keravel
olonkinbyen - nearest hotel: No hotel found
balkanabat - nearest hotel: Nebitçi
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
puerto natales - nearest hotel: Dorotea Patagonia Hostel
ribeira grande - nearest hotel: Hospedaria JSF
rio grande - nearest hotel: Hotel Vila Moura Executivo
valparaiso - nearest hotel: Ibis Hotel
iqaluit - nearest hotel: Frobisher Inn
awjilah - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
saint-apollinaire - nearest hotel: Campanile Dijon Est
broken hill - nearest hotel: Ibis Styles
bethel - nearest hotel: Hampton Inn Danbury
edinburgh of the seven seas - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
east london - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
fort bragg - nearest hotel: Airborne Inn Lodging
sirte - nearest hotel: فندق المدينة-سرت
albany - nearest hotel: No hotel found
hermanus - neares

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
   'Lng', 'Lat',
    hover_cols=['Hotel Name', 'City'],
    color='blue',
    size=20,
    geo=True,
    title='Hotel Locations',
    tiles="OSM"
)
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City)